In [11]:
# Import necessary libraries
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import networkx as nx
import os, sys

# Workaround for __file__ not being defined in Jupyter notebooks
notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '../..')))  # Adjusted path

from engine.core_rec import *  

In [12]:

# Import custom modules
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data

# Workaround for __file__ not being defined in Jupyter notebooks
notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '../..')))


# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Convert data to torch tensors
x = data.x
edge_index = data.edge_index

# Create a PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index)

# Add train_mask and test_mask
num_nodes = x.size(0)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:int(0.8 * num_nodes)] = True
test_mask = ~train_mask

data.train_mask = train_mask
data.test_mask = test_mask

# Add labels (y) if not already present
num_classes = dataset.num_classes
data.y = data.y if data.y is not None else torch.randint(0, num_classes, (num_nodes,), dtype=torch.long)


In [13]:
adj_matrix = nx.to_numpy_array(nx.from_edgelist(edge_index.t().tolist()))
train_dataset = GraphDataset(adj_matrix[train_mask])
test_dataset = GraphDataset(adj_matrix[test_mask])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:

# Initialize the model
num_layers = 1
d_model = 128
num_heads = 8
d_feedforward = 512
input_dim = 2708 # Ensure this matches the number of features in the dataset
model = GraphTransformerV2(num_layers, d_model, num_heads, d_feedforward, input_dim)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=100)
# Evaluate the model
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, targets in test_loader:
        # print(f"Input shape: {inputs.shape}")  # Add this line to check the input shape
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        # print(f"Preds shape: {preds.shape}, Targets shape: {targets.shape}")  # Debugging shapes
        targets = torch.argmax(targets, dim=1)  # Convert one-hot encoded targets to class indices
        all_preds.extend(preds.cpu().numpy().flatten())  # Ensure preds are flattened
        all_labels.extend(targets.cpu().numpy().flatten())  # Ensure targets are flattened
        # print(f"Current length of all_preds: {len(all_preds)}, all_labels: {len(all_labels)}")  # Debugging lengths

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Debugging: Print final lengths of all_preds and all_labels
# print(f"Final length of all_preds: {len(all_preds)}")
# print(f"Final length of all_labels: {len(all_labels)}")

# Ensure both arrays are 1-dimensional
all_preds = all_preds.flatten()
all_labels = all_labels.flatten()

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

Epoch 1/100, Loss: 23.88054847717285
Epoch 2/100, Loss: 15.703673362731934
Epoch 3/100, Loss: 18.538469314575195
Epoch 4/100, Loss: 16.13208770751953
Epoch 5/100, Loss: 17.79981231689453
Epoch 6/100, Loss: 53.00846862792969
Epoch 7/100, Loss: 11.9574556350708
Epoch 8/100, Loss: 6.425912857055664
Epoch 9/100, Loss: 10.688874244689941
Epoch 10/100, Loss: 8.965605735778809
Epoch 11/100, Loss: 12.352456092834473
Epoch 12/100, Loss: 8.098556518554688
Epoch 13/100, Loss: 8.64078140258789
Epoch 14/100, Loss: 9.981680870056152
Epoch 15/100, Loss: 8.952320098876953
Epoch 16/100, Loss: 17.278013229370117
Epoch 17/100, Loss: 8.168448448181152
Epoch 18/100, Loss: 12.569095611572266
Epoch 19/100, Loss: 10.038256645202637
Epoch 20/100, Loss: 8.95732593536377
Epoch 21/100, Loss: 7.703975200653076
Epoch 22/100, Loss: 9.584030151367188
Epoch 23/100, Loss: 10.356860160827637
Epoch 24/100, Loss: 12.819650650024414
Epoch 25/100, Loss: 12.636499404907227
Epoch 26/100, Loss: 8.94835090637207
Epoch 27/100, L

In [38]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.datasets import Planetoid
import matplotlib.pyplot as plt

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define a simple GAT model
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=8):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define the GraphTransformer model (assuming the class is already defined)
# from your previous implementation
# class GraphTransformer(torch.nn.Module):
#     # Your GraphTransformer implementation
#     pass

# Define a simple MLP model
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x = data.x
        x = self.fc1(x).relu()
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
num_epochs=200
# Function to train a model
if model == 'GraphTransformer':
    num_layers = 1
    d_model = 128
    num_heads = 8
    d_feedforward = 512
    input_dim = 2708
    # model = GraphTransformerV2(num_layers, d_model, num_heads, d_feedforward, input_dim)
    train_model(model, train_loader, criterion, optimizer, num_epochs=100)
else:
    lr=0.01
    weight_decay=5e-4
    def train_model123(model, data, num_epochs, criterion=criterion, optimizer=optimizer):
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Function to evaluate a model
def evaluate_model(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    return acc

# Load data
data = dataset[0]

# Debugging: Print data attributes
print(f"Data attributes: {data.keys}")

# Debugging: Print data types
print(f"Data x type: {type(data.x)}, Data edge_index type: {type(data.edge_index)}")

# Ensure data is in the correct format
data.x = data.x.float()
data.edge_index = data.edge_index.long()

# Define models
input_dim = dataset.num_node_features
hidden_dim = 16
output_dim = dataset.num_classes

import sys, os
notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '../..')))
from engine.core_rec import *

models = {
    'GCN': GCN(input_dim, hidden_dim, output_dim),
    'GAT': GAT(input_dim, hidden_dim, output_dim),
    'MLP': MLP(input_dim, hidden_dim, output_dim),
    # Add your GraphTransformer here with the appropriate parameters
    # 'GraphTransformer': GraphTransformer(num_layers=2, d_model=hidden_dim, num_heads=2, d_feedforward=hidden_dim, input_dim=input_dim)
}

# Train and evaluate models
accuracies = {}
for name, model in models.items():
    print(f"Training {name}...")
    if model == 'GraphTransformer':
        train_model(model, train_loader, criterion, optimizer, num_epochs=100)
    else:
        train_model(model, data, num_epochs)  # Corrected function call
    acc = evaluate_model(model, data)
    accuracies[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

# Plot accuracies
plt.figure(figsize=(10, 6))
plt.bar(accuracies.keys(), accuracies.values(), color='skyblue')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model Accuracies on Cora Dataset')
plt.show()

AttributeError: 'GlobalStorage' object has no attribute 'float'

In [23]:
from torch.nn import Module, Linear, TransformerEncoderLayer, TransformerEncoder, ReLU, ModuleList
from torch_geometric.utils import degree
import torch



class GraphTransformer(Module):
    def __init__(self, num_layers, d_model, num_heads, d_feedforward, input_dim, num_weights=10, use_weights=True):
        super(GraphTransformer, self).__init__()
        self.num_weights = num_weights
        self.use_weights = use_weights
        self.input_linear = Linear(input_dim, d_model)
        self.encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=d_feedforward, batch_first=True)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.output_linear = Linear(d_model, input_dim)
        if self.use_weights:
            self.weight_linears = ModuleList([Linear(input_dim, d_model) for _ in range(num_weights)])

    def forward(self, x, weights=None):
        x = x.float()
        if self.use_weights:
            if weights is not None:
                weighted_x = torch.zeros_like(x)
                for i, weight in enumerate(weights):
                    weighted_x += self.weight_linears[i](x) * weight
                x = weighted_x
            else:
                x = self.input_linear(x)
        else:
            x = self.input_linear(x)
        x = self.transformer_encoder(x)
        x = self.output_linear(x)
        return x

